In [76]:
import pandas as pd

# Read Metadata
fer2013 = pd.read_csv('/Users/brensikeg/Desktop/Final/archive/fer2013/fer2013.csv')

emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

fer2013 = fer2013.rename(columns={"emotion":"FER label"})
fer2013['FER Emotion'] = fer2013['FER label'].apply(lambda x: emotions[int(x)])

fer2013_new = pd.read_csv('/Users/brensikeg/Desktop/Final/archive/fer2013/fer2013new.csv')
fer2013_new.drop(['Usage', 'Image name', 'Contempt', 'Unknown', 'NF'], inplace=True, axis=1)

meta_df = pd.concat([fer2013, fer2013_new], axis=1, join='inner')

In [77]:
meta_df

,FER label,pixels,Usage,FER Emotion,Angry,Disgust,Fear,Happy,Sad,Surprise,Neutral
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training,Angry,3,2,0,0,1,0,4
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training,Angry,0,0,0,0,1,1,6
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training,Fear,1,0,0,0,3,0,5
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training,Sad,1,0,0,0,4,0,4
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training,Neutral,0,0,0,0,1,0,9
...,...,...,...,...,...,...,...,...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest,Neutral,0,0,0,0,2,0,8
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest,Happy,0,0,0,0,0,0,0
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest,Angry,7,1,0,0,0,0,0
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest,Happy,0,0,0,10,0,0,0


In [78]:
df = meta_df.iloc[:,4:]
df_result = pd.DataFrame(df.max(axis=1), columns=['value'])

def get_col_name(row):    
    b = (df.loc[row.name] == row['value'])
    return b.index[b.argmax()]

def get_col_label(row):    
    b = (df.loc[row.name] == row['value'])
    return b.argmax()

meta_df['FER+ label'] = df_result.apply(get_col_label, axis=1)
meta_df['FER+ Emotion'] = df_result.apply(get_col_name, axis=1)
meta_df.drop(emotions, inplace=True, axis=1)

In [79]:
meta_df

,FER label,pixels,Usage,FER Emotion,FER+ label,FER+ Emotion
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training,Angry,6,Neutral
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training,Angry,6,Neutral
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training,Fear,6,Neutral
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training,Sad,4,Sad
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training,Neutral,6,Neutral
...,...,...,...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest,Neutral,6,Neutral
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest,Happy,0,Angry
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest,Angry,0,Angry
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest,Happy,3,Happy


In [83]:
import dlib
import cv2
from imutils import face_utils
import math

hogFaceDetector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/Users/brensikeg/Desktop/Final/shape_predictor_68_face_landmarks.dat")

def getLandmarks(df):
    features = []
    
    for image in df:
        gray = image.astype('uint8')
        faces = hogFaceDetector(gray, 1)
        
        for (i, rect) in enumerate(faces):
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

        features.append(shape)
    
    new_features = []
    
    for row in features:
        means = row.mean(axis=0)

        tmp = []
    
        for pt in row:
            mag = math.sqrt((means[0]-pt[0])*(means[0]-pt[0]) + (means[1]-pt[1])*(means[1]-pt[1]))
            bearing = math.atan((means[1]-pt[1])/(means[0]-pt[0]))
        
            tmp.append([pt[0], pt[1], mag, bearing])
    
        tmp = np.concatenate(tmp, axis=0)
    
        new_features.append(tmp)
        
    return new_features

In [84]:
import pandas as pd
import numpy as np

def process(df, filename):
    pix_list = df['pixels'].tolist()
    pixels = np.array([np.fromstring(x, dtype=int, sep=' ') for x in pix_list])
    pixels = np.reshape(pixels, (-1, 48, 48))

    with open(filename + '_pixels.npy', 'wb') as f:
        np.save(f, pixels)

    lab_list = df['FER+ label'].tolist()
    labels = np.asarray(lab_list)

    with open(filename + '_labels.npy', 'wb') as f:
        np.save(f, labels)
   
    landmarks = getLandmarks(pixels)

    with open(filename + '_landmarks.npy', 'wb') as f:
        np.save(f, landmarks)

train_df = meta_df[meta_df["Usage"] == "Training"]
eval_df  = meta_df[meta_df["Usage"] == "PublicTest"]
test_df  = meta_df[meta_df["Usage"] == "PrivateTest"]

process(train_df, 'data/train')
process(eval_df, 'data/eval')
process(test_df, 'data/test')

<ipython-input-83-1522485c7566>:31: RuntimeWarning: divide by zero encountered in double_scalars
  bearing = math.atan((means[1]-pt[1])/(means[0]-pt[0]))
